In [87]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np
from keras import layers
import numpy as np
import spacy
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [22]:
df = pd.read_json('./cleaned_kaggle_data.json', encoding='utf-8', lines=True)

In [32]:
df['category'] = df['category'].replace("U.S. NEWS", "US NEWS")

In [36]:
df.head()

,link,headline,category,short_description,authors,date,cleaned_text,text,category_encoded
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,US NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,4 million Americans roll sleeve Omicron target...,Over 4 Million Americans Roll Up Sleeves For O...,31
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",US NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,American Airlines Flyer charge ban life Punchi...,"American Airlines Flyer Charged, Banned For Li...",31
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Funniest tweet Cats Dogs Week Sept. 17 23 d...,23 Of The Funniest Tweets About Cats And Dogs ...,4
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,Funniest Tweets parent Week Sept. 17 23 accide...,The Funniest Tweets From Parents This Week (Se...,20
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,US NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,woman call cop Black Bird Watcher lose Lawsuit...,Woman Who Called Cops On Black Bird-Watcher Lo...,31


In [41]:
le = LabelEncoder()
df['category_encoded'] = le.fit_transform(df['category'])

In [43]:
# Remember the encoding values
le.classes_

array(['ARTS & CULTURE', 'BLACK VOICES', 'BUSINESS', 'COLLEGE', 'COMEDY',
       'CRIME', 'DIVORCE', 'EDUCATION', 'ENTERTAINMENT', 'ENVIRONMENT',
       'FIFTY', 'FOOD & DRINK', 'GOOD NEWS', 'GREEN', 'HEALTHY LIVING',
       'HOME & LIVING', 'IMPACT', 'LATINO VOICES', 'MEDIA', 'MONEY',
       'PARENTING', 'PARENTS', 'POLITICS', 'QUEER VOICES', 'RELIGION',
       'SCIENCE', 'SPORTS', 'STYLE & BEAUTY', 'TASTE', 'TECH', 'TRAVEL',
       'US NEWS', 'WEDDINGS', 'WEIRD NEWS', 'WELLNESS', 'WOMEN',
       'WORLD NEWS', 'WORLDPOST'], dtype=object)

In [42]:
df.head()

,link,headline,category,short_description,authors,date,cleaned_text,text,category_encoded
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,US NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23,4 million Americans roll sleeve Omicron target...,Over 4 Million Americans Roll Up Sleeves For O...,31
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",US NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23,American Airlines Flyer charge ban life Punchi...,"American Airlines Flyer Charged, Banned For Li...",31
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23,23 Funniest tweet Cats Dogs Week Sept. 17 23 d...,23 Of The Funniest Tweets About Cats And Dogs ...,4
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23,Funniest Tweets parent Week Sept. 17 23 accide...,The Funniest Tweets From Parents This Week (Se...,20
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,US NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22,woman call cop Black Bird Watcher lose Lawsuit...,Woman Who Called Cops On Black Bird-Watcher Lo...,31


In [97]:
nlp = spacy.load('en_core_web_lg')

In [98]:
unique_words = set()
df['cleaned_text'].apply(lambda x: unique_words.update([token.text.lower() for token in nlp(x) if token.is_alpha]))
vocab_size = len(unique_words)
vocab_size

In [79]:
y = df['category_encoded']
X = df['cleaned_text']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [82]:
vocab_size = 1000
encoder = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
encoder.adapt(X_train)


In [85]:
vocab = np.array(encoder.get_vocabulary())
len(vocab)

1000

In [74]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim= vocab_size + 1, output_dim=64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [75]:
sample_text = ('The movie was coole and the graphis were out of this world')
predictions = model.predict(np.array([sample_text]))

ValueError: Invalid dtype: str1856